# Intro to DL Day 2: NLP
We'll start with an introduction to different NLP tasks and how to use already pre-trained algorithms to perform these tasks using the [Huggingface](https://huggingface.co) library (most popular library in NLP these days) !! (This notebook is therefore greatly inspired by their [courses](https://huggingface.co/course/chapter1).
Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.9 MB/s eta 0:00:00


# NLP tasks
## 1.1 Sentiment classification

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a Deep Learning course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.8671445250511169}]

We can even pass in multiple sentences

In [3]:
classifier(
    ["I've been waiting for a Deep Learning course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.8671445250511169},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

By default, this pipeline selects a particular pretrained model that has been fine-tuned for sentiment analysis in English. The model is downloaded and cached when you create the classifier object. If you rerun the command, the cached model will be used instead and there is no need to download the model again.

There are three main steps involved when you pass some text to a pipeline:

1. The text is preprocessed into a format the model can understand.
1. The preprocessed inputs are passed to the model.
1. The predictions of the model are post-processed, so you can make sense of them.

Some of the currently available pipelines are:

- feature-extraction (get the vector representation of a text)
- fill-mask
- ner (named entity recognition)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification

Let’s have a look at a few of these!

## 1.2  Zero-shot classification
We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled. This is a common scenario in real-world projects because annotating text is usually time-consuming and requires domain expertise. For this use case, the zero-shot-classification pipeline is very powerful: it allows you to specify which labels to use for the classification, so you don’t have to rely on the labels of the pretrained model. You’ve already seen how the model can classify a sentence as positive or negative using those two labels — but it can also classify the text using any other set of labels you like.

In [4]:
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course using the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course using the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8637737035751343, 0.09807975590229034, 0.038146551698446274]}

This pipeline is called zero-shot because you don’t need to fine-tune the model on your data to use it. It can directly return probability scores for any list of labels you want!

In zero shot classification, we provide the model with a prompt and a sequence of text that describes what we want our model to do, in natural language. Zero-shot classification excludes any examples of the desired task being completed. This differs from single or few-shot classification, as these tasks include a single or a few examples of the selected task.

Zero, single and few-shot classification seem to be an emergent feature of large language models. This feature seems to come about around model sizes of +100M parameters. The effectiveness of a model at a zero, single or few-shot task seems to scale with model size, meaning that larger models (models with more trainable parameters or layers) generally do better at this task.

It will basically ask to the model a question like:

```
Classify the following input text into one of the following three categories: [education, business, politics]

Input Text: This is a course using the Transformers library
Category: Education
```

In [5]:
# try it out yourself with some examples

## 1.3 Text generation

Now let’s see how to use a pipeline to generate some text. The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text. This is similar to the predictive text feature that is found on many phones. Text generation involves randomness, so it’s normal if you don’t get the same results as shown below.

In [6]:
generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1187: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1278: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'In this course, we will teach you how to build an object-oriented programming experience by combining concepts from the game engine and the game framework. The goal is to build programs that run as programmable objects in a manner that allows them to communicate and'}]

You can control how many different sequences are generated with the argument `num_return_sequences` and the total length of the output text with the argument `max_length`.

In [7]:
generator("In this course, we will teach you how to", num_return_sequences=3, max_length=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to use the Microsoft Visual Studio 2010 Runtime Environment commandline in Microsoft Office 365 and enable a simple, step-by-step process to deploy your.WP applications to Windows Server 2016. Using this guide,'},
 {'generated_text': 'In this course, we will teach you how to set up a user-facing website, making website based on cookies. When not creating a user-friendly website, you must first create one (for example, to generate a search form). And first'},
 {'generated_text': 'In this course, we will teach you how to make a simple, concise Python program called "python code generator."\n\nA quick introduction to Python code generator.\n\nIn this course, you will start with a brief introduction to the Python programming'}]

You can as well specify what model you want to use in your pipeline:

In [8]:
generator = pipeline("text-generation", model="distilgpt2")
generator("In this course, we will teach you how to")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "In this course, we will teach you how to teach the following techniques.\n\n\n\nNow in today's course the following skills might be shown:\nThe first basic question is how to teach each new thing. Let's take a look at"}]

## 1.4 Mask filling
The idea of this task is to fill in the blanks in a given text:

In [9]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'score': 0.1961982548236847,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052723944187164,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

The `top_k` argument controls how many possibilities you want to be displayed. Note that here the model fills in the special `<mask>` word, which is often referred to as a mask token. Other mask-filling models might have different mask tokens, so it’s always good to verify the proper mask word when exploring other models.

## 1.5  Named entity recognition
Named entity recognition (NER) is a task where the model has to find which parts of the input text correspond to entities such as persons, locations, or organizations. Let’s look at an example:

In [10]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Thomas and I work at Business & Decision in Brussels.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


/opt/conda/lib/python3.7/site-packages/transformers/pipelines/token_classification.py:160: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  "`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to"


[{'entity_group': 'PER',
  'score': 0.99934226,
  'word': 'Thomas',
  'start': 11,
  'end': 17},
 {'entity_group': 'ORG',
  'score': 0.99677676,
  'word': 'Business & Decision',
  'start': 32,
  'end': 51},
 {'entity_group': 'LOC',
  'score': 0.99659485,
  'word': 'Brussels',
  'start': 55,
  'end': 63}]

We pass the option grouped_entities=True in the pipeline creation function to tell the pipeline to regroup together the parts of the sentence that correspond to the same entity: here the model correctly grouped “Business”, "&" and “Decision” as a single organization, even though the name consists of multiple words. In fact, as we will see in the next chapter, the preprocessing even splits some words into smaller parts. In the post-processing step, the pipeline successfully regrouped those pieces.

## 1.6 Question answering

The question-answering pipeline answers questions using information from a given context:

In [11]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Thomas and I work at Business & Decision in Brussels",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.6524998545646667,
 'start': 32,
 'end': 51,
 'answer': 'Business & Decision'}

## 1.7 Summarization

Summarization is the task of reducing a text into a shorter text while keeping all (or most) of the important aspects referenced in the text. Here’s an example:

In [12]:
summarizer = pipeline("summarization")
summarizer(
    """
Business & Decision, part of Orange Group, is one of the world's leading management consultancies and system integrators for Data Intelligence & Digital Experience.  

We are Data Native Artists who leverage a unique combination of technical, functional and industry specialization, as well as partnerships with key software vendors, to deliver state of the art solutions since 1992.     

As a front runner in Big Data, Artificial Intelligence and Digital, Business & Decision is enabling customers to innovate, drive their business strategy and improve customer experience through effective use of data.   

Clients choose Business & Decision as their strategic Data & Digital partner due to our pioneering vision, expertise, core values, quality of service & our passion for delivery.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': " Business & Decision, part of Orange Group, is one of the world's leading management consultancies and system integrators for Data Intelligence & Digital Experience . It is enabling customers to\u202finnovate,\u202fdrive their business strategy and improve customer experience through effective use of data ."}]

Like with text generation, you can specify a `max_length` or a `min_length` for the result.

## 1.8 Translation

For translation, you can use a default model if you provide a language pair in the task name (such as "translation_en_to_fr"), but the easiest way is to pick the model you want to use on the Model Hub. Here we’ll try translating from French to English:

In [13]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours utilise le package python crée par Hugging Face.")

/opt/conda/lib/python3.7/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'This course uses the python package created by Hugging Face.'}]